In [1]:
import time

note = """
    khi decorator cho method
    
    def tracer(func): #1                   
        
        # argument first nen =self de hinh dung hon la chi co *args
        # *args da co self trong do
        
        def wrapper(self, *args, **kwargs): #2
            wrapper.count+=1
            print(f"tracer.wrapper {wrapper.count}")
            nonlocal calls
            calls += 1
            print('call %s to %s' % (calls, func.__name__))

            return func(self, *args, **kwargs) #2

        wrapper.count = 0
        return wrapper #1
    
"""

def tracer(func): #1                   
    calls = 0
    print(f"tracer")
    
    def wrapper(*args, **kwargs): #2
        wrapper.count+=1
        print(f"tracer.wrapper {wrapper.count}")
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        
        return func(*args, **kwargs) #2
    
    wrapper.count = 0
    return wrapper #1



def timer(label='', trace=True): #1 parameter
    print("timer")
    
    def decorator(func): #2
        decorator.count+=1
        print(f"timer.decorator: {decorator.count}")
        
        def wrapper(*args, **kargs): #3
            wrapper.count+=1
            print(f"timer.decorator.wrapper {wrapper.count}")
            start   = time.time()  
            result  = func(*args, **kargs)
            elapsed = time.time() - start
            wrapper.alltime += elapsed
            
            if trace:
                format = '%s%s: %.5f, %.5f'
                values = (label, func.__name__, elapsed, wrapper.alltime)
                print(format % values)
                
            return result #3
        
        wrapper.count = 0
        wrapper.alltime = 0
        
        return wrapper #2
    
    decorator.count = 0
    return decorator #1

In [2]:
class Person:
    
    @tracer
    def __init__(self, name, pay):
        self.name = name
        self.pay  = pay

    @tracer
    def giveRaise(self, percent):         # giveRaise = tracer(giverRaise)
        self.pay *= (1.0 + percent)
    
    @tracer
    def lastName(self):                   # lastName = tracer(lastName)
        return self.name.split()[-1]

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)
print()

print(bob.lastName(), sue.lastName()) 

tracer
tracer
tracer
tracer.wrapper 1
call 1 to __init__
tracer.wrapper 2
call 2 to __init__

Bob Smith Sue Jones

tracer.wrapper 1
call 1 to giveRaise
110000.00

tracer.wrapper 1
call 1 to lastName
tracer.wrapper 2
call 2 to lastName
Smith Jones


## 1. decorateAll

In [3]:
from types import FunctionType

def decorateAll(decorator): #1
    print("decorateAll")
    
    def wrapper(cls): #2
        print("decorateAll.wrapper")
              
        for funcname, func in cls.__dict__.items():
            if type(func) is FunctionType:
                setattr(cls, funcname, decorator(func))
                 # cls.funcname = decorator(func)
        
        return cls #2
    
    return wrapper #1


@decorateAll(tracer)          # Person = decorateAll(tracer)(Person) => Person              
class Person:            
    
    def __init__(self, name, pay):            
        self.name = name
        self.pay  = pay
    
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
    
    def lastName(self):
        return self.name.split()[-1]

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)

print()
print(bob.lastName(), sue.lastName())

decorateAll
decorateAll.wrapper
tracer
tracer
tracer
tracer.wrapper 1
call 1 to __init__
tracer.wrapper 2
call 2 to __init__

Bob Smith Sue Jones

tracer.wrapper 1
call 1 to giveRaise
110000.00

tracer.wrapper 1
call 1 to lastName
tracer.wrapper 2
call 2 to lastName
Smith Jones


## 2. decorateAll

In [4]:
def decorateAll(decorator): #1 decorator parameter
    
    def wrapper(cls): #2
        for funcname, func in cls.__dict__.items():
            if type(func) is FunctionType:
                setattr(cls, funcname, decorator(func))
                 # cls.funcname = decorator(func)
                 # @decorator cho tat ca function cu class
        
        return cls #2
    
    return wrapper #1

#@decorateAll(tracer)  

@decorateAll(timer(label='@@'))  # Person = decorateAll(tracer)(Person) = Person
class Person:                         
    
    def __init__(self, name, pay):            
        self.name = name            
        self.pay  = pay
    
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
    
    def lastName(self):
        return self.name.split()[-1]

bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)

print()
print(bob.lastName(), sue.lastName())

print('-'*40)
print('%.5f' % Person.__init__.alltime)
print('%.5f' % Person.giveRaise.alltime)
print('%.5f' % Person.lastName.alltime)

timer
timer.decorator: 1
timer.decorator: 2
timer.decorator: 3
timer.decorator.wrapper 1
@@__init__: 0.00001, 0.00001
timer.decorator.wrapper 2
@@__init__: 0.00001, 0.00002

Bob Smith Sue Jones

timer.decorator.wrapper 1
@@giveRaise: 0.00001, 0.00001
110000.00

timer.decorator.wrapper 1
@@lastName: 0.00001, 0.00001
timer.decorator.wrapper 2
@@lastName: 0.00001, 0.00002
Smith Jones
----------------------------------------
0.00002
0.00001
0.00002


In [5]:
import time

def tracer(func): #1                   
    calls = 0
    print(f"tracer")
    
    def wrapper(*args, **kwargs): #2
        wrapper.count+=1
        print(f"tracer.wrapper {wrapper.count}")
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        
        return func(*args, **kwargs) #2
    
    wrapper.count = 0
    return wrapper #1


def timer(label='', trace=True): #1 parameter
    print("timer")
    
    def decorator(func): #2
        decorator.count+=1
        print(f"timer.decorator: {decorator.count}")
        
        def wrapper(*args, **kargs): #3
            wrapper.count+=1
            print(f"timer.decorator.wrapper {wrapper.count}")
            start   = time.time()  
            result  = func(*args, **kargs)
            elapsed = time.time() - start
            wrapper.alltime += elapsed
            
            if trace:
                format = '%s%s: %.5f, %.5f'
                values = (label, func.__name__, elapsed, wrapper.alltime)
                print(format % values)
                
            return result #3
        
        wrapper.count = 0
        wrapper.alltime = 0
        
        return wrapper #2
    
    decorator.count = 0
    return decorator #1


#@decorateAll(tracer)                          # Use a class decorator
#@decorateAll(tracer(timer(label='@@')))  

#@decorateAll(timer(label='@@')(tracer))      # Times applying the tracer!
#@decorateAll(tracer(timer(label='@@')))      # Traces applying the timer!

#@decorateAll(tracer)                          # Traces onCall wrapper, times methods
#@decorateAll(timer(label='@@'))

#@decorateAll(timer(label='@@'))
#@decorateAll(tracer)                          # Times onCall wrapper, traces methods

       
@decorateAll(timer(label='@@'))       # Person = decorateAll(timer(label='@@'))(Person)
@decorateAll(tracer)                  # Person = decorateAll(tracer)(Person) = Person
class Person:                   
    
    def __init__(self, name, pay):          
        self.name = name
        self.pay  = pay
    
    def giveRaise(self, percent):
        self.pay *= (1.0 + percent)
    
    def lastName(self):
        return self.name.split()[-1]
    
print(f"{'':*^50}")
print()
    
bob = Person('Bob Smith', 50000)
sue = Person('Sue Jones', 100000)
print()

print(bob.name, sue.name)
print()

sue.giveRaise(.10)
print('%.2f' % sue.pay)

print()
print(bob.lastName(), sue.lastName())


timer
tracer
tracer
tracer
timer.decorator: 1
timer.decorator: 2
timer.decorator: 3
**************************************************

timer.decorator.wrapper 1
tracer.wrapper 1
call 1 to __init__
@@wrapper: 0.00007, 0.00007
timer.decorator.wrapper 2
tracer.wrapper 2
call 2 to __init__
@@wrapper: 0.00006, 0.00013

Bob Smith Sue Jones

timer.decorator.wrapper 1
tracer.wrapper 1
call 1 to giveRaise
@@wrapper: 0.00007, 0.00007
110000.00

timer.decorator.wrapper 1
tracer.wrapper 1
call 1 to lastName
@@wrapper: 0.00006, 0.00006
timer.decorator.wrapper 2
tracer.wrapper 2
call 2 to lastName
@@wrapper: 0.00006, 0.00012
Smith Jones
